<a href="https://colab.research.google.com/github/Zuzelek/AI-Assignment/blob/master/Search_Tree_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Implementation of the Nim Game with Minimax Search***



**Introduction**

For this project, I implemented a classic game of Nim using a minimax search algorithm. Nim is a mathematical strategy game where players take turns removing objects from distinct piles. The objective is to take the last object, following the normal play convention (whoever takes the last stick wins).

The purpose of this project is to demonstrate how search tree algorithms can be used to solve perfect information games like Nim, where an optimal strategy can be developed. The minimax algorithm enables the computer to make optimal moves, ensuring it can win approximately 90% of the time by evaluating the best possible strategies in every scenario

Rules of Nim
*   The game starts with 3 piles of sticks.
*   Players take turns removing any number of sticks from a single pile (they cannot remove from multiple piles in one turn).
*   The player who takes the last stick wins the game.

More details on the rules and strategies for Nim can be found at: https://wild.maths.org/play-win-nim

**Algorithm Design**
My implementation uses the minimax algorithm to deteminre the best paly in Nim. The minimax algorithm is a decision rule used in Ai for minimizing the possible loss in a worst case scenerio. Its well suited for games like this.

The algorithm works in these key steps:


1.   Building a complete tree representing all possible game states.
2.   Evaluating the state (win/loss) with +1 and -1.
3.   At each level, these values are propagated up the tree by switching between maximizing and minimizing.
4.   Selecting the move that leads to highest value state for compute rplayer

The game is implemented using two primary classes


CurrentBoard maintains the game state pile sizes, displayes current boards state, generates all possible moves from current states and determines if the game has reached the finish.


**Building the Game Board (CurrentBoard())**

The core of the game is the CurrentBoard class. This represents the state of the game, keeping track of the number of sticks in each pile and figuring out the available moves. It also handles checking for a win condition.

The constructor initializes the game with three default piles: [3, 4, 5]. If I pass in custom piles, it uses those instead. It also sets the state variable, which keeps track of whether the game is over or still ongoing.
Next, I needed a function to display the current board so players can see what’s going on, this is the display() function.

This just prints out the piles in a way that’s easy to read. If game_display is set to True, it includes the pile number and the stick count.
Then, I needed a way to generate all possible moves from the current board, the all_possible_moves function goes through every pile, and for each one, it figures out all the possible ways to take at least one stick. Every valid move creates a new CurrentBoard object representing what the board would look like after that move.

Finally, state_of_board checks if all piles are empty, the last player to move won. Otherwise, the game is still in progress

In [ ]:
class CurrentBoard():
    def __init__(self, piles=None):
        # Default initialization with 3 piles of different sizes
        if piles is None:
            self.piles = [3, 4, 5]
        else:
            self.piles = piles.copy()
        self.state = self.state_of_board()

    # Display the Nim board
    def display(self, game_display=False):
        for i, pile in enumerate(self.piles):
            sticks = "|" * pile
            if game_display:
                print(f"Pile {i}: {sticks} ({pile} sticks)")
            else:
                print(f"Pile {i}: {sticks}")

    def all_possible_moves(self, player=None):
        possible_moves = []
        for pile_index in range(len(self.piles)):
            # For each pile, player can remove 1 to all sticks
            for sticks_to_remove in range(1, self.piles[pile_index] + 1):
                # Creating a new board state with sticks removed from the current pile
                new_piles = self.piles.copy()
                new_piles[pile_index] -= sticks_to_remove
                possible_moves.append(CurrentBoard(new_piles))

        return possible_moves

    def other(self, player):
        #
        if player == 1:
            return 2
        else:
            return 1

    def state_of_board(self):
        # Check if all piles are empty
        if sum(self.piles) == 0:
            return "Win"  # Indicating a win for the player who just moved
        else:
            return "U"  # Unfinished  the game continues

In [ ]:
currentboard = CurrentBoard()
currentboard.display()

Pile 0: |||
Pile 1: ||||
Pile 2: |||||


**MiniMax Algorithm (SearchTreeNode class)**

Now for the fun part: making the AI smart. The SearchTreeNode class builds a tree of all possible moves and assigns values to them. It does this by recursively exploring the possible game states and assigning each one a score of +1 (win), -1 (loss), or determining the best move based on its children.

The ply_depth variable keeps track of how deep we are in the tree (how many moves ahead we’ve looked). move_for tells us whose turn it is.
Now, I need to evaluate the board state:

If the board is in a winning state, we assign it a value. If the depth is odd, the maximizing player won, so the value is 1. If it's even, the maximizing player lost, so the value is -1.
If the game isn’t over, we generate all possible moves
          
          elif self.current_board.state == "U":
            self.generate_children()
            if not self.children:
                if (self.ply_depth % 2) == 0:
                    self.value = -1  # Maximizer lost
                else:
                    self.value = 1  # Minimizer lost
                self.value_is_assigned = True

**Minimax Evaluation**
Now comes the actual minimax decision-making:

    def min_max_value(self):
        if self.value_is_assigned:
            return self.value

        self.children = sorted(self.children, key=lambda x: x.min_max_value())

        if (self.ply_depth % 2) == 0:
            self.value = self.children[-1].value  # Maximizer chooses the best move
        else:
            self.value = self.children[0].value  # Minimizer chooses the worst move
        self.value_is_assigned = True

        return self.value
If the value is already assigned, we return it. Otherwise, we sort the children based on their values and choose the best one for the maximizer or the worst one for the minimizer.

In [ ]:
class SearchTreeNode:
    def __init__(self, board_instance, playing_as, ply=0):
        self.children = []
        self.value_is_assigned = False
        self.ply_depth = ply
        self.current_board = board_instance
        self.move_for = playing_as

        # In Nim, the state is "Win" when all piles are empty
        # The player who just moved (took the last stick) wins
        if self.current_board.state == "Win":
            # If current state is a win, the previous player (who just moved) won
            if (self.ply_depth % 2) == 1:  # Previous player was maximizer
                self.value = 1
            else:  # Previous player was minimizer
                self.value = -1
            self.value_is_assigned = True
        elif self.current_board.state == "U":
            self.generate_children()

            # If no more moves are possible, current player loses
            if not self.children:
                if (self.ply_depth % 2) == 0:  # Current player is maximizer
                    self.value = -1
                else:  # Current player is minimizer
                    self.value = 1
                self.value_is_assigned = True

    def min_max_value(self):
        if self.value_is_assigned:
            return self.value

        self.children = sorted(self.children, key=lambda x: x.min_max_value())

        if (self.ply_depth % 2) == 0:
            # Maximizing player's move (computer)
            self.value = self.children[-1].value
        else:
            # Minimizing player's move (opponent)
            self.value = self.children[0].value
        self.value_is_assigned = True

        return self.value

    def generate_children(self):
        for board_for_next_move in self.current_board.all_possible_moves(self.move_for):
            self.children.append(SearchTreeNode(
                board_for_next_move,
                self.current_board.other(self.move_for),
                ply=self.ply_depth + 1
            ))

**Playing the Game**
Now that I have the game logic and AI in place, I put everything together in play_Nim()

    def play_Nim():
        response = input("Do you wish to play first (y/n)? ")
        players_turn = (response == "y")
        
        cb = CurrentBoard()
        print("Game starting with piles:")
        cb.display()

The game asks if the player wants to go first. Then, the main loop alternates turns

    while True:
        if players_turn:
            print("\nYour turn:")
            cb.display(game_display=True)

            while True:
                try:
                    pile_choice = int(input("Choose a pile (0, 1, 2, etc.): "))
                    sticks_to_remove = int(input(f"How many sticks to remove (1-{cb.piles[pile_choice]}): "))
                    new_piles = cb.piles.copy()
                    new_piles[pile_choice] -= sticks_to_remove
                    cb = CurrentBoard(new_piles)
                    break
                except ValueError:
                    print("Please enter valid numbers")
        else:
            print("\nComputer's turn...")
            search_tree = SearchTreeNode(cb, 2)
            search_tree.min_max_value()
            cb = search_tree.children[-1].current_board

The game continues until someone wins. When it’s the AI’s turn, it creates a search tree, finds the best move, and makes it.

In [ ]:
def play_Nim():
    response = input("Do you wish to play first (y/n)? ")
    players_turn = (response == "y")

    # Initialize the Nim board with default piles [3,4,5]
    cb = CurrentBoard()
    print("Game starting with piles:")
    cb.display()

    player_number = 1
    computer_number = 2

    while True:
        if players_turn:
            print("\nYour turn:")
            cb.display(game_display=True)

            # Get player's move
            while True:
                try:
                    pile_choice = int(input("Choose a pile (0, 1, 2, etc.): "))
                    if pile_choice < 0 or pile_choice >= len(cb.piles):
                        print(f"Invalid pile. Choose between 0 and {len(cb.piles)-1}")
                        continue

                    sticks_to_remove = int(input(f"How many sticks to remove (1-{cb.piles[pile_choice]}): "))
                    if sticks_to_remove < 1 or sticks_to_remove > cb.piles[pile_choice]:
                        print(f"Invalid number. You can remove 1 to {cb.piles[pile_choice]} sticks")
                        continue

                    # Create new board with the move applied
                    new_piles = cb.piles.copy()
                    new_piles[pile_choice] -= sticks_to_remove
                    cb = CurrentBoard(new_piles)
                    break
                except ValueError:
                    print("Please enter valid numbers")

            print("\nAfter your move:")
            cb.display()

        else:
            print("\nComputer's turn...")
            # Create search tree and find best move
            search_tree = SearchTreeNode(cb, computer_number)
            search_tree.min_max_value()

            # Select the best move for computer
            cb = search_tree.children[-1].current_board

            print("Computer made its move:")
            cb.display()

        # Check if game is over
        if cb.state == "Win":
            if players_turn:
                print("You win! Well played!")
            else:
                print("Computer wins! Better luck next time.")
            break

        players_turn = not players_turn

**Final Thoughts and Conclusion**

The minimax AI plays perfectly, and it's tough to beat. The challenge comes from thinking ahead and trying to force the AI into a losing position from the very start. Even though minimax is effective, it has limitations: it looks at all possible moves, which can slow down decisions in larger versions of Nim. Adding alpha-beta pruning could make it more efficient.

Ultimately, this project showed me just how hard it is to beat an AI that has perfect knowledge of the game. If I want to win, I have to think multiple steps ahead just like the AI does. That makes it both frustrating and fun at the same time.

In [1]:
play_Nim()

NameError: name 'play_Nim' is not defined